## DL C-deep Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'cind'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'cind'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.34329035046734807
Concordance Index Test 0.3924249037736598
{'model': 'cind', 'dataset': 'METABRIC', 'cindex_train': [0.34329035046734807], 'cindex_test': [0.3924249037736598], 'ibs_train': [nan], 'ibs_score_test': [nan]}
1
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.33808695956914475
Concordance Index Test 0.3635273063383744
{'model': 'cind', 'dataset': 'METABRIC', 'cindex_train': [0.33808695956914475], 'cindex_test': [0.3635273063383744], 'ibs_train': [nan], 'ibs_score_test': [nan]}
2
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.337328981109817
Concordance Index Test 0.35802787304482286
{'model': 'cind', 'dataset': 'METABRIC', 'cindex_t

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,cind,METABRIC,0.343290,0.392425,NaN,NaN
1,cind,METABRIC,0.338087,0.363527,NaN,NaN
2,cind,METABRIC,0.337329,0.358028,NaN,NaN
3,cind,METABRIC,0.355032,0.336474,NaN,NaN
4,cind,METABRIC,0.349297,0.336875,NaN,NaN
5,cind,FLCHAIN,0.203095,0.192236,NaN,NaN
6,cind,FLCHAIN,0.202140,0.190728,NaN,NaN
7,cind,FLCHAIN,0.197835,0.213788,NaN,NaN
8,cind,FLCHAIN,0.196396,0.218809,NaN,NaN
9,cind,FLCHAIN,0.201740,0.198462,NaN,NaN


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'cind'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.22161584129208303
Concordance Index Test 0.32320441988950277
{'model': 'cind', 'dataset': 'BLCA', 'cindex_train': [0.22161584129208303], 'cindex_test': [0.32320441988950277], 'ibs_train': [nan], 'ibs_score_test': [nan]}
1
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.30417362270450754
Concordance Index Test 0.4336428181321682
{'model': 'cind', 'dataset': 'BLCA', 'cindex_train': [0.30417362270450754], 'cindex_test': [0.4336428181321682], 'ibs_train': [nan], 'ibs_score_test': [nan]}
2
model: cind
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.15041268462206778
Concordance Index Test 0.32974910394265233
{'model': 'cind', 'dataset': 'BLCA', 'cindex_train': [0.15

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)